# import libraries

In [76]:
import joblib
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

# Data Overview

In [77]:
df = pd.read_csv('english.csv')

In [78]:
df.head(10)

,image,label
0,Img/img001-001.png,0
1,Img/img001-002.png,0
2,Img/img001-003.png,0
3,Img/img001-004.png,0
4,Img/img001-005.png,0
5,Img/img001-006.png,0
6,Img/img001-007.png,0
7,Img/img001-008.png,0
8,Img/img001-009.png,0
9,Img/img001-010.png,0


it contain two columns:
- image -> path for the images 
- label -> the digit or character of the images

In [79]:
df.shape

(3410, 2)

the dataset contains **``55 images``** for every character or digit

## size of the images

In [80]:
img_size = (32, 32)

## load and preprocess images

In [81]:
def load_and_preprocess_image(image_path):
    img = load_img(image_path, target_size=img_size, color_mode='grayscale')
    return img_to_array(img) / 255.0 

In [82]:
image_paths = df['image'].values
labels = df['label'].values

In [83]:
# Encode character labels to integers
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)

# preprocess images

In [84]:
imgs = np.array([load_and_preprocess_image(img_path) for img_path in image_paths])

In [85]:
num_label = len(np.unique(labels_encoded))
one_hot_labels = np.eye(num_label)[labels_encoded]

# split data

In [86]:
X_train, X_val, y_train, y_val = train_test_split(imgs, one_hot_labels, test_size=0.1, random_state=42)

# Data augmentation

In [87]:
datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1
)
datagen.fit(X_train)

# modeling

In [88]:
# Define the model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 1)),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(num_label, activation='softmax')
])

C:\Users\Mohammed237\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [89]:
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [90]:
# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_model.keras', save_best_only=True, monitor='val_loss')
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.00001)

In [91]:
# Train the model with data augmentation
history = model.fit(
    datagen.flow(X_train, y_train, batch_size=32),
    epochs=50,
    validation_data=(X_val, y_val),
    callbacks=[early_stopping, model_checkpoint, reduce_lr]
)

Epoch 1/50
 5/96 ━━━━━━━━━━━━━━━━━━━━ 2s 27ms/step - accuracy: 0.0130 - loss: 6.3803

C:\Users\Mohammed237\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


96/96 ━━━━━━━━━━━━━━━━━━━━ 5s 25ms/step - accuracy: 0.0340 - loss: 4.6385 - val_accuracy: 0.0176 - val_loss: 6.3228 - learning_rate: 0.0010
Epoch 2/50
96/96 ━━━━━━━━━━━━━━━━━━━━ 2s 25ms/step - accuracy: 0.1136 - loss: 3.6280 - val_accuracy: 0.0176 - val_loss: 8.5070 - learning_rate: 0.0010
Epoch 3/50
96/96 ━━━━━━━━━━━━━━━━━━━━ 2s 23ms/step - accuracy: 0.2091 - loss: 3.0873 - val_accuracy: 0.0176 - val_loss: 7.9136 - learning_rate: 0.0010
Epoch 4/50
96/96 ━━━━━━━━━━━━━━━━━━━━ 3s 30ms/step - accuracy: 0.3161 - loss: 2.4975 - val_accuracy: 0.0528 - val_loss: 4.8093 - learning_rate: 0.0010
Epoch 5/50
96/96 ━━━━━━━━━━━━━━━━━━━━ 3s 28ms/step - accuracy: 0.4047 - loss: 2.1103 - val_accuracy: 0.2639 - val_loss: 2.7882 - learning_rate: 0.0010
Epoch 6/50
96/96 ━━━━━━━━━━━━━━━━━━━━ 2s 25ms/step - accuracy: 0.4663 - loss: 1.8265 - val_accuracy: 0.6070 - val_loss: 1.5766 - learning_rate: 0.0010
Epoch 7/50
96/96 ━━━━━━━━━━━━━━━━━━━━ 2s 24ms/step - accuracy: 0.5339 - loss: 1.5629 - val_accuracy: 0.54

In [95]:
# Evaluate the model
loss, accuracy = model.evaluate(X_val, y_val)
print(f'Validation Loss: {loss}')
print(f'Validation Accuracy: {accuracy}')

11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7875 - loss: 0.6867 
Validation Loss: 0.6027182340621948
Validation Accuracy: 0.8093841671943665


# Saving model and label encoder

In [93]:
model.save('english_character_recognition_model.keras')

In [94]:
# Save the label encoder
joblib.dump(label_encoder, 'label_encoder.pkl')


['label_encoder.pkl']